In [4]:
pip install voila


Note: you may need to restart the kernel to use updated packages.


# Introduzione alla Fisica dello Stato Solido

## 1. Concetti Fondamentali
La fisica dello stato solido è una branca della fisica della materia condensata che studia le proprietà dei materiali solidi in relazione alla loro struttura atomica. L'ordine cristallino, le simmetrie del reticolo e le interazioni tra gli atomi determinano caratteristiche fondamentali come proprietà elettriche, magnetiche, ottiche e meccaniche.

Gli sviluppi nel campo della fisica dello stato solido hanno portato a numerose innovazioni tecnologiche, tra cui i semiconduttori, i materiali superconduttori e le nanotecnologie. Le proprietà dei solidi dipendono dalla natura dei legami chimici tra gli atomi, dalla loro disposizione spaziale e dalle interazioni con fattori esterni come temperatura, pressione e campi elettrici o magnetici.

## 2. Struttura Cristallina e Reticoli di Bravais
Un materiale cristallino è caratterizzato da una struttura atomica ordinata e periodica. Questa periodicità è descritta dal **reticolo cristallino**, un insieme di punti distribuiti nello spazio secondo un pattern regolare. A ciascun punto reticolare è associata una base di atomi che definisce la struttura cristallina del materiale.

La disposizione spaziale degli atomi nei solidi può essere rappresentata da una **cella unitaria**, il più piccolo volume ripetitivo che descrive la struttura del cristallo. Il **vettore reticolare** è espresso come combinazione lineare dei vettori primitivi:

$$
\mathbf{R} = n_1 \mathbf{a}_1 + n_2 \mathbf{a}_2 + n_3 \mathbf{a}_3
$$

dove $n_1, n_2, n_3$ sono numeri interi e $\mathbf{a}_1, \mathbf{a}_2, \mathbf{a}_3$ sono i vettori primitivi del reticolo.



## 3. Indici di Miller
Per descrivere piani e direzioni all'interno di un cristallo si utilizzano gli **indici di Miller**, che rappresentano la relazione tra il piano considerato e gli assi cristallografici. Un piano è identificato da tre numeri interi $(hkl)$, ottenuti prendendo gli inversi delle intersezioni del piano con gli assi e riducendoli ai minimi termini.

Il metodo per determinare gli indici di Miller è il seguente:

1. Determinare le intersezioni del piano con gli assi cristallografici in unità della cella unitaria $(x, y, z)$.
2. Calcolare gli inversi di questi valori.
3. Moltiplicare per un minimo comune multiplo per ottenere numeri interi.

Ad esempio, se un piano interseca gli assi nei punti $(2,3,1)$, gli indici di Miller si ottengono come:

$$
\frac{1}{2}, \frac{1}{3}, \frac{1}{1} \Rightarrow (3,2,6)
$$

Gli indici di Miller sono fondamentali per comprendere la disposizione degli atomi nei cristalli e per analizzare fenomeni come la diffrazione.

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown

# Función para generar el retículo en 2D
def generate_2d_lattice(angle_deg, lattice_type, size):
    angle_rad = np.radians(angle_deg)
    
    # Vectores de la celda unitaria para cada tipo de retículo
    lattices = {
        "Cuadrado": np.array([[1, 0], [0, 1]]),
        "Rectangular": np.array([[1.5, 0], [0, 1]]),
        "Oblicuo": np.array([[1, 0], [np.cos(angle_rad), np.sin(angle_rad)]]),
        "Hexagonal": np.array([[1, 0], [0.5, np.sqrt(3) / 2]])
    }
    
    if lattice_type not in lattices:
        raise ValueError(f"Lattice type '{lattice_type}' no soportado.")
    
    # Obtener los vectores base del retículo
    base_vectors = lattices[lattice_type]
    
    # Convertir size a entero
    size = int(size)
    
    # Generar la red
    points = []
    for i in range(-size, size + 1):
        for j in range(-size, size + 1):
            point = i * base_vectors[0] + j * base_vectors[1]
            points.append(point)
    points = np.array(points)
    
    return points, base_vectors

# Función para dibujar las conexiones entre puntos
def draw_connections(ax, points, base_vectors, lattice_type):
    if lattice_type in ["Cuadrado", "Rectangular", "Oblicuo"]:
        for point in points:
            neighbors = [
                point + base_vectors[0],  # Vecino en dirección del primer vector base
                point - base_vectors[0],
                point + base_vectors[1],  # Vecino en dirección del segundo vector base
                point - base_vectors[1]
            ]
            for neighbor in neighbors:
                if any(np.allclose(neighbor, p) for p in points):
                    ax.plot([point[0], neighbor[0]], [point[1], neighbor[1]], 'k-', lw=0.5)
    
    elif lattice_type == "Hexagonal":
        hex_directions = [
            base_vectors[0], 
            -base_vectors[0],
            base_vectors[1], 
            -base_vectors[1],
            base_vectors[0] - base_vectors[1],
            base_vectors[1] - base_vectors[0]
        ]
        for point in points:
            for direction in hex_directions:
                neighbor = point + direction
                if any(np.allclose(neighbor, p) for p in points):
                    # Solo conectar puntos en los bordes, no el punto central
                    if np.linalg.norm(direction) < 1.5:  # Distancia máxima para conexiones
                        ax.plot([point[0], neighbor[0]], [point[1], neighbor[1]], 'k-', lw=0.5)

# Función para visualizar el retículo en 2D
def plot_2d_lattice(angle, lattice_type, size):
    points, base_vectors = generate_2d_lattice(angle, lattice_type, size)
    
    # Crear la figura
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.scatter(points[:, 0], points[:, 1], c='blue', s=50, label="Puntos de red")
    
    # Dibujar las conexiones
    draw_connections(ax, points, base_vectors, lattice_type)
    
    # Dibujar vectores base si aplica
    if lattice_type == "Oblicuo":
        ax.plot([0, base_vectors[0, 0]], [0, base_vectors[0, 1]], 'r-', lw=2, label="Vector base 1")
        ax.plot([0, base_vectors[1, 0]], [0, base_vectors[1, 1]], 'g-', lw=2, label="Vector base 2")
        ax.text(0.5 * base_vectors[0, 0], 0.5 * base_vectors[0, 1], f"{angle}°", color="red", fontsize=12)
    
    # Configurar los ejes y el gráfico
    ax.axhline(0, color='black', linewidth=0.5)
    ax.axvline(0, color='black', linewidth=0.5)
    ax.set_aspect('equal', adjustable='box')
    ax.set_title(f"Retículo {lattice_type} con ángulo {angle}°", fontsize=14)
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.5)
    plt.show()

# Widget interactivo
interact(
    plot_2d_lattice,
    angle=FloatSlider(value=90, min=0, max=180, step=1, description="Ángulo (°):"),
    lattice_type=Dropdown(options=["Cuadrado", "Rectangular", "Oblicuo", "Hexagonal"], value="Cuadrado", description="Tipo de retículo:"),
    size=FloatSlider(value=5, min=2, max=10, step=1, description="Tamaño:")
)


interactive(children=(FloatSlider(value=90.0, description='Ángulo (°):', max=180.0, step=1.0), Dropdown(descri…

<function __main__.plot_2d_lattice(angle, lattice_type, size)>

Le celle elementari sono le unità fondamentali che descrivono la periodicità del reticolo. Tra queste, la **cella di Wigner-Seitz** è una scelta particolare che suddivide lo spazio in regioni attorno a ciascun punto reticolare, in modo da includere tutti i punti più vicini a un dato sito reticolare rispetto agli altri.

I **reticoli di Bravais** classificano tutte le possibili disposizioni periodiche degli atomi nello spazio tridimensionale. Esistono 14 tipi di reticolo di Bravais, suddivisi in sette sistemi cristallini:

- Cubico (semplice, a corpo centrato, a facce centrate)
- Tetragonale (semplice, a corpo centrato)
- Ortorombico (semplice, a corpo centrato, a facce centrate, base centrata)
- Romboedrico
- Esagonale
- Monoclino (semplice, base centrata)
- Triclino

In [1]:
%matplotlib ipympl  

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# Diccionario de los 14 retículos organizados por sistemas cristalinos
reticulos_bravais = {
    "Cúbico": ["Cúbico simple (SC)", "Cúbico centrado en el cuerpo (BCC)", "Cúbico centrado en las caras (FCC)"],
    "Tetragonal": ["Tetragonal simple", "Tetragonal centrado en el cuerpo"],
    "Ortorrómbico": ["Ortorrómbico simple", "Ortorrómbico centrado en el cuerpo", "Ortorrómbico centrado en caras", "Ortorrómbico centrado en todas las caras"],
    "Hexagonal": ["Hexagonal"],
    "Trigonal": ["Trigonal"],
    "Monoclínico": ["Monoclínico simple", "Monoclínico centrado en caras"],
    "Triclínico": ["Triclínico"]
}

# Función para generar los vértices y aristas de cada retículo de Bravais
def generar_retículo(tipo):
    if tipo == "Cúbico simple (SC)":
        puntos = np.array([[0, 0, 0], [3, 0, 0], [3, 3, 0], [0, 3, 0],  
                           [0, 0, 3], [3, 0, 3], [3, 3, 3], [0, 3, 3]])
        aristas = [
            (0,1), (1,2), (2,3), (3,0),  # Base inferior
            (4,5), (5,6), (6,7), (7,4),  # Base superior
            (0,4), (1,5), (2,6), (3,7)   # Aristas verticales
        ]
    
    elif tipo == "Cúbico centrado en el cuerpo (BCC)":
        puntos = np.array([[0, 0, 0], [3, 0, 0], [3, 3, 0], [0, 3, 0],  
                           [0, 0, 3], [3, 0, 3], [3, 3, 3], [0, 3, 3],  
                           [1.5, 1.5, 1.5]])
        aristas = [
            (0,1), (1,2), (2,3), (3,0),  # Base inferior
            (4,5), (5,6), (6,7), (7,4),  # Base superior
            (0,4), (1,5), (2,6), (3,7),  # Aristas verticales
            (8,0), (8,1), (8,2), (8,3),  # Conexiones con el centro
            (8,4), (8,5), (8,6), (8,7)
        ]
    
    elif tipo == "Cúbico centrado en las caras (FCC)":
        puntos = np.array([[0, 0, 0], [3, 0, 0], [3, 3, 0], [0, 3, 0],  
                           [0, 0, 3], [3, 0, 3], [3, 3, 3], [0, 3, 3],  
                           [1.5, 0, 1.5], [3, 1.5, 1.5], [1.5, 3, 1.5], [0, 1.5, 1.5],  
                           [1.5, 1.5, 0] , [1.5, 1.5, 3]])
        aristas = [
            (0,1), (1,2), (2,3), (3,0),  # Base inferior
            (4,5), (5,6), (6,7), (7,4),  # Base superior
            (0,4), (1,5), (2,6), (3,7),  # Aristas verticales
            (8,0), (8,4), (8,5), (8,1),  # Centro a vértices
            (9,1), (9,5), (9,6), (9,2),
            (10,2), (10,6), (10,7), (10,3),
            (11,3), (11,7), (11,4), (11,0),  
            (12,0), (12,1), (12,2), (12,3),
            (13,4), (13,5), (13,6), (13,7)
        ]
    
    elif tipo == "Tetragonal simple":
        puntos = np.array([[0, 0, 0], [1, 0, 0], [1, 1, 0], [0, 1, 0],  
                           [0, 0, 2], [1, 0, 2], [1, 1, 2], [0, 1, 2]])
        aristas = [
            (0,1), (1,2), (2,3), (3,0),  # Base inferior
            (4,5), (5,6), (6,7), (7,4),  # Base superior
            (0,4), (1,5), (2,6), (3,7)   # Aristas verticales
        ]
    
    elif tipo == "Tetragonal centrado en el cuerpo":
        puntos = np.array([[0, 0, 0], [1, 0, 0], [1, 1, 0], [0, 1, 0],  
                           [0, 0, 2], [1, 0, 2], [1, 1, 2], [0, 1, 2],  
                           [0.5, 0.5, 1]])
        aristas = [
            (0,1), (1,2), (2,3), (3,0),  # Base inferior
            (4,5), (5,6), (6,7), (7,4),  # Base superior
            (0,4), (1,5), (2,6), (3,7),  # Aristas verticales
            (8,0), (8,1), (8,2), (8,3),  # Conexiones con el centro
            (8,4), (8,5), (8,6), (8,7)
        ]

    elif tipo == "Ortorrómbico simple":
        puntos = np.array([[0, 0, 0], [1, 0, 0], [1, 2, 0], [0, 2, 0],  # Base inferior
            [0, 0, 3], [1, 0, 3], [1, 2, 3], [0, 2, 3]])  # Base superior)
        aristas = [
            (0,1), (1,2), (2,3), (3,0),  # Base inferior
            (4,5), (5,6), (6,7), (7,4),  # Base superior
            (0,4), (1,5), (2,6), (3,7)   # Aristas verticales
        ]

    elif tipo == "Ortorrómbico centrado en el cuerpo":
        puntos = np.array([[0, 0, 0], [1, 0, 0], [1, 2, 0], [0, 2, 0],  # Base inferior
            [0, 0, 3], [1, 0, 3], [1, 2, 3], [0, 2, 3],  # Base superior
            [0.5, 1, 1.5]])
        aristas = [
            (0, 1), (1, 2), (2, 3), (3, 0),  # Base inferior
            (4, 5), (5, 6), (6, 7), (7, 4),  # Base superior
            (0, 4), (1, 5), (2, 6), (3, 7),  # Aristas verticales
            (8, 0), (8, 1), (8, 2), (8, 3), 
            (8,4), (8,5), (8,6), (8,7) # Conexión con la cara inferior 
        ]
    
    elif tipo == "Ortorrómbico centrado en caras":
        puntos = np.array([
        [0, 0, 0], [1, 0, 0], [1, 2, 0], [0, 2, 0],  # Base inferior
        [0, 0, 3], [1, 0, 3], [1, 2, 3], [0, 2, 3],  # Base superior
        [0.5, 1, 0], [0.5, 1, 3]  # Centros de las caras superior e inferior 
        ])
        aristas = [
        (0, 1), (1, 2), (2, 3), (3, 0),  # Base inferior
        (4, 5), (5, 6), (6, 7), (7, 4),  # Base superior
        (0, 4), (1, 5), (2, 6), (3, 7),  # Aristas verticales
        (8, 0), (8, 1), (8, 2), (8, 3),  # Conexión con la cara inferior
        (9, 5), (9, 6), (9, 7), (9, 4)   # Conexión con la cara superior
        ]
    
    elif tipo == "Ortorrómbico centrado en todas las caras":
        puntos = np.array([
            [0, 0, 0], [1, 0, 0], [1, 2, 0], [0, 2, 0],  # Base inferior
            [0, 0, 3], [1, 0, 3], [1, 2, 3], [0, 2, 3],  # Base superior
            [0.5, 0, 1.5], [1, 1, 1.5], [0.5, 2, 1.5], [0, 1, 1.5],  # Centros de las caras
            [0.5, 1, 0], [0.5, 1, 3]  # Otros puntos de conexión
        ])
        
        aristas = [
            (0, 1), (1, 2), (2, 3), (3, 0),  # Base inferior
            (4, 5), (5, 6), (6, 7), (7, 4),  # Base superior
            (0, 4), (1, 5), (2, 6), (3, 7),  # Aristas verticales
            (8, 0), (8, 4), (8, 5), (8, 1),  # Conexiones con caras
            (9, 1), (9, 5), (9, 6), (9, 2),
            (10, 2), (10, 6), (10, 7), (10, 3),
            (11, 3), (11, 7), (11, 4), (11, 0),
            (12, 0), (12, 1), (12, 2), (12, 3),
            (13, 4), (13, 5), (13, 6), (13, 7)  # Conexiones adicionales
        ]
        
    elif tipo == "Hexagonal":
        puntos = np.array([
            [3, 0, 2], [2.5, 0, 2 + np.sqrt(3)/2], [1.5, 0, 2 + np.sqrt(3)/2], 
            [1, 0, 2], [1.5, 0, 2 - np.sqrt(3)/2], [2.5, 0, 2 - np.sqrt(3)/2], 
            [3, 2, 2], [2.5, 2, 2 + np.sqrt(3)/2], [1.5, 2, 2 + np.sqrt(3)/2], 
            [1, 2, 2], [1.5, 2, 2 - np.sqrt(3)/2], [2.5, 2, 2 - np.sqrt(3)/2], 
            [2, 0, 2], [2, 2, 2]
        ])

        aristas = [
            (0, 1), (1, 2), (2, 3), (3, 4),  # Aristas de la cara inferior
            (4, 5), (5, 0), (6, 7), (7, 8),  # Aristas de la cara superior
            (8, 9), (9, 10), (10, 11), (11, 6),  # Aristas en la cara superior
            (0, 6), (1, 7), (2, 8), (3, 9), (4, 10), (5, 11),  # Aristas verticales
            (0, 12), (1, 12), (2, 12), (3, 12), (4, 12), (5, 12),  # Aristas desde el centro del primer hexágono
            (6, 13), (7, 13), (8, 13), (9, 13), (10, 13), (11, 13)  # Aristas desde el centro del segundo hexágono
        ]


    elif tipo == "Trigonal":
        puntos = np.array([
    [1, 1, 1],        # A
    [2.3, 1, 1],      # B
    [2.6, 2.2, 1],    # C
    [1.2, 2.3, 1],    # D
    [1, 1, 2.3],      # A'
    [2.3, 1, 2.4],    # B'
    [2.6, 2.2, 2.5],  # C'
    [1.2, 2.3, 2.4]   # D'
    ])

        aristas = [
            (0, 1),  # A-B
        (1, 2),  # B-D
        (3, 2),  # D-C
        (3, 0),  # C-A
        (4, 5),  # A'-B'
        (5, 6),  # B'-D'
        (7, 6),  # D'-C'
        (7, 4),  # C'-A'
        (0, 4),  # A-A'
        (1, 5),  # B-B'
        (2, 6),  # C-C'
        (3, 7)
        ]

    elif tipo == "Monoclínico simple":
        puntos = np.array([
    [0, 0, 0],        # A
    [1.2856, 0, 1.532],  # B (Inclinación en xz)
    [1.2856, 1.5, 1.532], # C
    [0, 1.5, 0],      # D
    [0, 0, 1],        # A'
    [1.2856, 0, 2.532],  # B'
    [1.2856, 1.5, 2.532],# C'
    [0, 1.5, 1]       # D'
        ])
        aristas = [
           (0, 1), (1, 2), (2, 3), (3, 0),  # 🔹 Base inferior
    (4, 5), (5, 6), (6, 7), (7, 4),  # 🔹 Base superior
    (0, 4), (1, 5), (2, 6), (3, 7) 
        ]

    elif tipo == "Monoclínico centrado en caras":
        puntos = np.array([[0, 0, 0],        # A
    [1.2856, 0, 1.532],  # B
    [1.2856, 1.5, 1.532], # C
    [0, 1.5, 0],      # D
    [0, 0, 1],        # A'
    [1.2856, 0, 2.532],  # B'
    [1.2856, 1.5, 2.532],# C'
    [0, 1.5, 1],      # D'
    [0, 0.75, 0.5], # Centro inferior
    [1.2856, 0.75, 2.032]])
        aristas = [
            (0,1), (1,2), (2,3), (3,0),  # Base inferior
            (4,5), (5,6), (6,7), (7,4),  # Base superior
            (0,4), (1,5), (2,6), (3,7),  # Aristas verticales
            (8,0), (8,3), (8,4), (8,7),
            (9,1), (9,2), (9,5), (9,6) # Conexiones con la cara
        ]
    
    elif tipo == "Triclínico":
        puntos = np.array([
        [0, 0, 0],         # A
        [2, 0, 0],         # B  (Lado de 2)
        [2.1, 1.5, 0.3],   # C  (Lado de 1.5 y ángulo distinto)
        [0.3, 1.6, 0.4],   # D  
        [0, 0, 1],         # A'
        [2, 0, 1.1],       # B'
        [2.1, 1.5, 1.4],   # C'
        [0.3, 1.6, 1.2]    # D'
    ])
    
        aristas = [
        (0,1), (1,2), (2,3), (3,0),  # 🔹 Base inferior
        (4,5), (5,6), (6,7), (7,4),  # 🔹 Base superior
        (0,4), (1,5), (2,6), (3,7),  # 🔹 Conexiones verticales
    ]
    return puntos, aristas

# Función de rotación de puntos (si se necesita)
def rotar_y(puntos, angulo):
    rot_mat = np.array([
        [np.cos(angulo), 0, np.sin(angulo)],
        [0, 1, 0],
        [-np.sin(angulo), 0, np.cos(angulo)]
    ])
    return np.dot(puntos, rot_mat.T)

# Interactividad para seleccionar tipo de retículo
@interact
def mostrar_reticulo(tipo=widgets.Dropdown(options=[val for sublist in reticulos_bravais.values() for val in sublist], description="Retículo:")):
    puntos, aristas = generar_retículo(tipo)
    
    # Crear la figura y el eje en 3D
    fig = plt.figure(figsize=(3, 3))
    ax = fig.add_subplot(111, projection='3d')

    ax.scatter(puntos[:, 0], puntos[:, 1], puntos[:, 2], c='r', marker='o')

    # Dibujar las aristas de acuerdo a las definiciones específicas de cada retículo
    for arista in aristas:
        ax.plot([puntos[arista[0], 0], puntos[arista[1], 0]], 
                [puntos[arista[0], 1], puntos[arista[1], 1]], 
                [puntos[arista[0], 2], puntos[arista[1], 2]], c='b')

    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    ax.set_title(tipo)
    
    # Establecer límites constantes para los ejes
    ax.set_xlim([0, 3])
    ax.set_ylim([0, 3])
    ax.set_zlim([0, 3])

    plt.show()

interactive(children=(Dropdown(description='Retículo:', options=('Cúbico simple (SC)', 'Cúbico centrado en el …

# Física dello Stato Solido: Cristallografia e Diffrazione

## 4. Studio della Struttura Cristallina
Lo studio della struttura cristallina avviene attraverso la diffrazione, inviando un fascio di particelle o radiazione con una lunghezza d'onda paragonabile a quella della reticolo. La condizione per avere un massimo di diffrazione è data dalla Legge di Bragg:

$$
2d \sin \theta = n \lambda
$$

Dove:

- **d** è la distanza tra i piani reticolari.
- **θ** è l'angolo di incidenza.
- **n** è un numero intero.
- **λ** è la lunghezza d'onda della radiazione incidente.

## 5. Teoria Cinematica della Diffrazione Elettronica
L'analisi della propagazione degli elettroni in un cristallo si divide in tre effetti principali:

- **Effetto del reticolo**: Ogni punto del reticolo agisce come un centro di dispersione delle onde, il che porta alla formulazione delle equazioni di Laue.
- **Effetto della base**: La presenza di più atomi in un punto del reticolo influisce sull'intensità della radiazione dispersa.
- **Effetto della distribuzione della carica atomica**: L'estensione finita degli atomi introduce un fattore di forma atomica che modifica l'ampiezza dell'onda dispersa.

### Equazioni di Laue
Per avere un massimo di diffrazione, l'ampiezza totale della radiazione dispersa sarà massima quando:

$$
a \cdot \Delta K = 2 \pi q
$$
$$
b \cdot \Delta K = 2 \pi r
$$
$$
c \cdot \Delta K = 2 \pi s
$$

Con **q**, **r**, **s** numeri interi.

Queste equazioni sono correlate al concetto di reticolo reciproco.

## 6. Reticolo Reciproco
Ogni reticolo reale definito dai vettori di traslazione **a**, **b**, **c** ha associato un reticolo reciproco, definito dai vettori:

$$
A = \frac{2 \pi}{V} (b \times c), \quad B = \frac{2 \pi}{V} (c \times a), \quad C = \frac{2 \pi}{V} (a \times b)
$$

Dove **V** è il volume della cella unitaria.


## 7. Condizioni per i Massimi di Diffrazione
Per diverse strutture cristalline, i massimi di diffrazione sono determinati dagli indici di Miller (**hkl**) e dalla disposizione degli atomi nella cella unitaria.

### Esempio: Cubo Semplice
La condizione di diffrazione è semplicemente:

$$
\sum (h x_j + k y_j + l z_j) = \text{intero}
$$

### Esempio: Cubico Centrado nel Corpo (BCC)
Per un sistema BCC, l'intensità di diffrazione è diversa a seconda che la somma **h + k + l** sia pari o dispari.

### Esempio: Cubico Centrado nelle Facce (FCC)
In una cella FCC, l'intensità di diffrazione varia a seconda che gli indici di Miller siano tutti pari o dispari.


In [ ]:
# Activar el backend interactivo de matplotlib
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from ipywidgets import interact, FloatSlider

# Función para calcular los vectores del retículo recíproco
def calcular_retículo_reciproco(a1, a2, a3):
    # Calcular el producto triple (volumen de la celda unitaria)
    volumen = np.dot(a1, np.cross(a2, a3))
    
    # Calcular los vectores del retículo recíproco
    b1 = 2 * np.pi * np.cross(a2, a3) / volumen
    b2 = 2 * np.pi * np.cross(a3, a1) / volumen
    b3 = 2 * np.pi * np.cross(a1, a2) / volumen
    
    return b1, b2, b3

# Función para graficar el retículo
def graficar_retículo(ax, vectores, color, label):
    # Crear los puntos de la celda unitaria
    puntos = np.array([
        [0, 0, 0],
        vectores[0],
        vectores[1],
        vectores[2],
        vectores[0] + vectores[1],
        vectores[0] + vectores[2],
        vectores[1] + vectores[2],
        vectores[0] + vectores[1] + vectores[2]
    ])
    
    # Definir las aristas de la celda unitaria
    aristas = [
        [puntos[0], puntos[1]],
        [puntos[0], puntos[2]],
        [puntos[0], puntos[3]],
        [puntos[1], puntos[4]],
        [puntos[1], puntos[5]],
        [puntos[2], puntos[4]],
        [puntos[2], puntos[6]],
        [puntos[3], puntos[5]],
        [puntos[3], puntos[6]],
        [puntos[4], puntos[7]],
        [puntos[5], puntos[7]],
        [puntos[6], puntos[7]]
    ]
    
    # Dibujar las aristas
    ax.add_collection3d(Line3DCollection(aristas, colors=color, label=label))

# Función principal para interactuar
def visualizar_retículo_reciproco(a1x, a1y, a1z, a2x, a2y, a2z, a3x, a3y, a3z):
    # Definir los vectores del retículo directo
    a1 = np.array([a1x, a1y, a1z])
    a2 = np.array([a2x, a2y, a2z])
    a3 = np.array([a3x, a3y, a3z])
    
    # Calcular los vectores del retículo recíproco
    b1, b2, b3 = calcular_retículo_reciproco(a1, a2, a3)
    
    # Crear la figura 3D
    fig = plt.figure(figsize=(10, 5))
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122, projection='3d')
    
    # Graficar el retículo directo
    graficar_retículo(ax1, [a1, a2, a3], color='blue', label='Retículo Directo')
    ax1.set_title("Retículo Directo")
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.legend()
    
    # Graficar el retículo recíproco
    graficar_retículo(ax2, [b1, b2, b3], color='red', label='Retículo Recíproco')
    ax2.set_title("Retículo Recíproco")
    ax2.set_xlabel('X')
    ax2.set_ylabel('Y')
    ax2.set_zlabel('Z')
    ax2.legend()
    
    # Ajustar los límites de los ejes
    for ax in [ax1, ax2]:
        ax.set_xlim([-5, 5])
        ax.set_ylim([-5, 5])
        ax.set_zlim([-5, 5])
    
    plt.tight_layout()
    plt.show()

# Crear sliders interactivos para los vectores del retículo directo
interact(visualizar_retículo_reciproco,
         a1x=FloatSlider(min=-5, max=5, step=0.1, value=1, description='a1x'),
         a1y=FloatSlider(min=-5, max=5, step=0.1, value=0, description='a1y'),
         a1z=FloatSlider(min=-5, max=5, step=0.1, value=0, description='a1z'),
         a2x=FloatSlider(min=-5, max=5, step=0.1, value=0, description='a2x'),
         a2y=FloatSlider(min=-5, max=5, step=0.1, value=1, description='a2y'),
         a2z=FloatSlider(min=-5, max=5, step=0.1, value=0, description='a2z'),
         a3x=FloatSlider(min=-5, max=5, step=0.1, value=0, description='a3x'),
         a3y=FloatSlider(min=-5, max=5, step=0.1, value=0, description='a3y'),
         a3z=FloatSlider(min=-5, max=5, step=0.1, value=1, description='a3z'))

### Proprietà del Reticolo Reciproco
I vettori del reticolo reciproco sono perpendicolari ai piani del reticolo reale con indici di Miller (**hkl**). La distanza interplanare è data da:

$$
d_{hkl} = \frac{2 \pi}{|G|}
$$

Con **G = hA + kB + lC**.

### Esempio: Reticolo Reciproco di un Cubo
Per un reticolo cubico semplice:

$$
A = \frac{2 \pi}{a} \hat{x}, \quad B = \frac{2 \pi}{a} \hat{y}, \quad C = \frac{2 \pi}{a} \hat{z}
$$

Questo significa che il reticolo reciproco di un cubico semplice è anch'esso un cubico semplice.


### Prima Zona di Brillouin
La cella di Wigner-Seitz nel reticolo reciproco è denominata prima zona di Brillouin. Per un cristallo cubico, la zona di Brillouin ha la forma di un ottaedro troncato.

## 8. Diffrazione dei Raggi X
La diffrazione dei raggi X permette di determinare la struttura dei cristalli mediante spettri XRD. I picchi di diffrazione dipendono da:

- **Dimensione dei cristalli** (**D_{hkl}**): Relazionata con l'equazione di Scherrer.
- **Microdeformazioni** (**β_e**).
- **Effetti strumentali**.

Il metodo di Williamson-Hall viene utilizzato per separare questi effetti attraverso l'equazione:

$$
\beta_{tot} \cos \theta = 4 \epsilon \sin \theta + \frac{K \lambda}{L}
$$

Dove:

- **ϵ** è la deformazione interna del cristallo.
- **L** è la dimensione media del cristallito.

# Caratteristiche dello spettro XRD da polveri

## Larghezza dei picchi di diffrazione

La **larghezza angolare** di un picco di diffrazione in uno spettro XRD da polveri è influenzata da diversi fattori, che possono essere suddivisi in due categorie principali:

- **Fattori intrinseci**: Dimensione dei cristalli e microdeformazioni (microstrain).
- **Fattori strumentali**: Relativi alla geometria di diffrazione e alle caratteristiche strumentali del dispositivo XRD.

## Effetto della dimensione dei cristalli

La **larghezza angolare del picco** di diffrazione è direttamente influenzata dalla dimensione dei cristalli. Secondo la **equazione di Scherrer**, la larghezza angolare (β) del picco di diffrazione per un cristallo di dimensione media **D(hkl)** è data da:

$
\beta = \frac{K \lambda}{D_{\text{hkl}} \cos \theta}
$

dove:
- $ \beta $ è la larghezza angolare del picco (in radianti),
- $ K $ è una costante di forma (tipicamente compresa tra 0.9 e 1),
- $ \lambda $ è la lunghezza d'onda della radiazione X incidente,
- $ D_{\text{hkl}} $ è la dimensione media del cristallo lungo i piani diffrangenti (hkl),
- $ \theta $ è l'angolo di Bragg.

Questa equazione mostra che se la dimensione del cristallo **D(hkl)** è maggiore, la larghezza del picco diminuirà, indicando cristalli più grandi con diffrangenti più stretti.

## Determinazione della dimensione e della deformazione di un cristallita

Quando sia la **dimensione del cristallo** che la **deformazione interna** sono presenti simultaneamente, possiamo calcolare entrambe utilizzando un modello di diffrazione che tiene conto di entrambi i contributi.

Il metodo per separare i contributi alla larghezza angolare del picco di diffrazione si basa sul principio che l'allargamento dimensionale (βL) e l'allargamento per deformazione (βe) variano in modo diverso in funzione dell'angolo di Bragg θ \theta :

- L'allargamento dimensionale (βL) varia come $ 1/\cos \theta $,
- L'allargamento per deformazione (βe) varia come $ \tan \theta $.

Quando entrambi i contributi sono presenti, la loro combinazione deve essere determinata dalla **convoluzione**. Per semplificare, il modello di **Williamson e Hall** assume che la convoluzione sia una somma semplice o una somma di quadrati, ottenendo la seguente relazione:

$
\beta_{\text{tot}}^2 = \beta_L^2 + \beta_e^2
$

Moltiplicando entrambi i membri per $ \cos \theta $, otteniamo:

$
\beta_{\text{tot}} \cos \theta = \frac{4 \varepsilon}{\lambda} + \frac{K}{L}
$

dove:
- $ \beta_{\text{tot}} $ è la larghezza totale del picco,
- $ \varepsilon $ è la deformazione interna del cristallo,
- $ L $ è la dimensione media del cristallito,
- $ K $ è una costante che dipende dalla geometria.

## Diagramma di Williamson-Hall

Per separare i contributi della deformazione e della dimensione del cristallo, tracciamo un grafico di βtotcosθ \beta_{\text{tot}} \cos \theta  rispetto a sinθ \sin \theta . Se tracciamo questa relazione, otteniamo una retta che può essere descritta dalla formula:

$
y = mx + c
$

dove:
- $ y = \beta_{\text{tot}} \cos \theta $,
- $ x = \sin \theta $,
- $ m = 4 \varepsilon $ (pendenza della retta, che fornisce la deformazione $ \varepsilon $),
- $ c = \frac{K}{L} $ (intercetta della retta, che fornisce la dimensione media $ L $), 

In questo modo, il diagramma di Williamson-Hall permette di determinare separatamente la **deformazione interna** e la **dimensione del cristallito**.